In [2]:
# ============================================
# 0) INSTALAR DEPENDÊNCIAS
# ============================================
!pip install -q "transformers>=4.40.0" "accelerate" "bitsandbytes" "gitpython" "pandas"

# ============================================
# 1) IMPORTS E CONFIGURAÇÕES BÁSICAS
# ============================================
from git import Repo
from pathlib import Path
import glob, json, re, ast
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

REPO_URL = "https://github.com/unclecode/crawl4ai"
REPO_DIR = Path("/content/crawl4ai_repo")

# ============================================
# 2) CLONAR REPOSITÓRIO
# ============================================
if REPO_DIR.exists():
    import shutil
    shutil.rmtree(REPO_DIR)

Repo.clone_from(REPO_URL, REPO_DIR)
print("✅ Repositório clonado em:", REPO_DIR)

# ============================================
# 3) FUNÇÃO PARA LER ARQUIVOS
# ============================================
def ler_arquivo(caminho):
    try:
        return Path(caminho).read_text(encoding="utf-8", errors="ignore")
    except Exception as e:
        print(f"Erro lendo {caminho}: {e}")
        return ""

# ============================================
# 4) SELECIONAR ARQUIVOS-ALVO (.py)
#    (Focamos só nos arquivos mais ricos em padrões)
# ============================================
alvos_manuais = [
    REPO_DIR / "crawl4ai" / "webhook.py",
    REPO_DIR / "crawl4ai" / "api.py",
    REPO_DIR / "crawl4ai" / "job.py",
    REPO_DIR / "crawl4ai" / "model_loader.py",
    REPO_DIR / "crawl4ai" / "async_dispatcher.py",  # se não existir, será ignorado
]

codigos = []
for f in alvos_manuais:
    if f.exists():
        conteudo = ler_arquivo(f)
        if len(conteudo.strip()) > 0:
            codigos.append((str(f), conteudo))

print(f"📄 Arquivos selecionados para análise: {len(codigos)}")
for f, _ in codigos:
    print(" -", f)

if not codigos:
    print("⚠️ Nenhum arquivo selecionado encontrado. Verifique caminhos.")

# ============================================
# 5) DIVIDIR EM CHUNKS (trechos menores)
# ============================================
MAX_CHARS = 2200   # menor pra garantir que não estoure contexto
chunks = []

for (arquivo, conteudo) in codigos:
    temp = conteudo
    while len(temp) > MAX_CHARS:
        chunks.append((arquivo, temp[:MAX_CHARS]))
        temp = temp[MAX_CHARS:]
    if temp.strip():
        chunks.append((arquivo, temp))

print(f"🧩 Total de chunks gerados:", len(chunks))

# ============================================
# 6) CARREGAR MODELO QWEN 2.5 CODER
# ============================================
MODEL_ID = "Qwen/Qwen2.5-Coder-0.5B-Instruct"

print("\n🤖 Carregando modelo:", MODEL_ID)
tok = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
mdl = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto",        # usa GPU se tiver, senão CPU
    trust_remote_code=True,
    torch_dtype="auto"
)
gen = pipeline("text-generation", model=mdl, tokenizer=tok)
print("✅ Modelo carregado!")

# ============================================
# 7) FUNÇÃO PARA EXTRAIR JSON DA RESPOSTA
# ============================================
def extrair_json(texto):
    """
    Tenta extrair um dicionário JSON válido de dentro do texto.
    """
    ini = texto.find("{")
    fim = texto.rfind("}")
    if ini != -1 and fim != -1 and fim > ini:
        bruto = texto[ini:fim+1]
    else:
        return None

    # tenta parse direto, com troca de aspas, e como literal python
    for parser in (
        lambda x: json.loads(x),
        lambda x: json.loads(x.replace("'", '"')),
        lambda x: ast.literal_eval(x)
    ):
        try:
            return parser(bruto)
        except Exception:
            pass

    return None

# ============================================
# 8) RODAR A IA EM CADA CHUNK (ANÁLISE DO CÓDIGO)
# ============================================
resultados = []

# começa com poucos chunks para não demorar demais
LIMIT_CHUNKS = min(8, len(chunks))  # até 8 blocos no máximo

print(f"\n🚀 Iniciando análise em {LIMIT_CHUNKS} chunks...\n")

for idx, (arquivo, trecho) in enumerate(chunks[:LIMIT_CHUNKS]):
    print(f"\n===== CHUNK {idx+1}/{LIMIT_CHUNKS} | Arquivo: {arquivo} =====\n")

    prompt = f"""
Você é um especialista em Engenharia de Software.
Analise o código abaixo e identifique PADRÕES DE PROJETO (Design Patterns)
e/ou PADRÕES ARQUITETURAIS usados.

Responda SOMENTE em JSON no formato:
{{
  "patterns":[
    {{"name":"<padrão>","type":"Design/Architecture","confidence":0.0,"evidence":"<trecho pequeno>"}}
  ],
  "notes":"<observações breves>"
}}

Arquivo: {arquivo}

Código:
{trecho}
"""

    saida = gen(
        prompt,
        max_new_tokens=320,   # menor para ficar mais rápido
        do_sample=False,
        temperature=0.0
    )[0]["generated_text"]

    # prévia só pra debug/visualização
    print("🔎 Prévia da resposta do modelo:")
    print(saida[:500], "\n")

    js = extrair_json(saida)
    if js:
        resultados.append(js)
    else:
        print("⚠️ Não foi possível extrair JSON deste chunk.")

print("\n🔍 JSONs válidos encontrados:", len(resultados))

# ============================================
# 9) CONSOLIDAR TODOS OS PADRÕES ENCONTRADOS
# ============================================
todos = []
for r in resultados:
    for p in r.get("patterns", []):
        todos.append(p)

if todos:
    df = pd.DataFrame(todos)
    print("\n📊 PADRÕES DETECTADOS (prévia):")
    print(df.head(15))
else:
    print("\n⚠️ Nenhum padrão consolidado. Verifique as respostas do modelo e o prompt.")

# ============================================
# 10) SALVAR RESULTADOS EM CSV E JSON
# ============================================
OUT = Path("/content/qwen_code_analysis")
OUT.mkdir(exist_ok=True)

if todos:
    csv_path = OUT / "patterns_from_code.csv"
    json_path = OUT / "patterns_from_code.json"

    df.to_csv(csv_path, index=False)
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(todos, f, ensure_ascii=False, indent=2)

    print("\n💾 Arquivos gerados:")
    print(" -", csv_path)
    print(" -", json_path)
else:
    print("\n❗Nenhum arquivo salvo porque não houve padrões consolidados.")


✅ Repositório clonado em: /content/crawl4ai_repo
📄 Arquivos selecionados para análise: 2
 - /content/crawl4ai_repo/crawl4ai/model_loader.py
 - /content/crawl4ai_repo/crawl4ai/async_dispatcher.py
🧩 Total de chunks gerados: 19

🤖 Carregando modelo: Qwen/Qwen2.5-Coder-0.5B-Instruct


Device set to use cpu


✅ Modelo carregado!

🚀 Iniciando análise em 8 chunks...


===== CHUNK 1/8 | Arquivo: /content/crawl4ai_repo/crawl4ai/model_loader.py =====

🔎 Prévia da resposta do modelo:

Você é um especialista em Engenharia de Software.
Analise o código abaixo e identifique PADRÕES DE PROJETO (Design Patterns)
e/ou PADRÕES ARQUITETURAIS usados.

Responda SOMENTE em JSON no formato:
{
  "patterns":[
    {"name":"<padrão>","type":"Design/Architecture","confidence":0.0,"evidence":"<trecho pequeno>"}
  ],
  "notes":"<observações breves>"
}

Arquivo: /content/crawl4ai_repo/crawl4ai/model_loader.py

Código:
from functools import lru_cache
from pathlib import Path
import subprocess, o 

⚠️ Não foi possível extrair JSON deste chunk.

===== CHUNK 2/8 | Arquivo: /content/crawl4ai_repo/crawl4ai/model_loader.py =====

🔎 Prévia da resposta do modelo:

Você é um especialista em Engenharia de Software.
Analise o código abaixo e identifique PADRÕES DE PROJETO (Design Patterns)
e/ou PADRÕES ARQUITETURAIS usados.

Re